In [ ]:
!pip install transformers[sentencepiece]==4.10.0 datasets --quiet


In [ ]:
!python -c  "import transformers;print(transformers.__version__)"

4.10.0


In [ ]:
!pip install -qq ipython-autotime

%load_ext autotime

time: 163 µs (started: 2021-09-03 10:14:39 +00:00)


In [ ]:
import random
import numpy as np
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf

        tf.random.set_seed(seed)

set_seed(1)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
!pip install wandb --quiet

import wandb
wandb.login()

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 170 kB 45.4 MB/s 
     |████████████████████████████████| 97 kB 6.8 MB/s 
     |████████████████████████████████| 133 kB 48.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

time: 32.7 s (started: 2021-09-03 09:08:35 +00:00)


In [ ]:
%env WANDB_PROJECT=OGBv_taskb

env: WANDB_PROJECT=OGBv_taskb
time: 3.57 ms (started: 2021-09-03 09:09:08 +00:00)


In [ ]:
!transformers-cli login

WARNING! `transformers-cli login` is deprecated and will be removed in v5. Please use `huggingface-cli login` instead.

        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: mlkorra
Password: 
Login successful
Your token: oqTSFfKSBzyUojayNVQFWNloHjSdSZMrVtawRTiNciFfFGYqRwQBzlFMIPclWWAdcwTPBTtoBXcIpgXUZEbGbXHUyJtJLgOMAaCwjCHAvrZmzxKMdgYjdhiklmcmoKrf 

Your token has been saved to /root/.huggingface/token
time: 18.6 s (started: 2021-09-03 09

In [ ]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "rohanrajpal/bert-base-en-hi-codemix-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/832 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

time: 2.6 s (started: 2021-09-03 09:09:30 +00:00)


In [ ]:
def labelencoder():

  """
  Creates and returns label encoders for each task.

  Parameters:
          task(str): Either Task 'A' or 'B' 

  Returns:
          task_labels (dict): label encoder dict of the given input task
  """
  
  task_a_labels = {'NAG':0,'OAG':1,'CAG':2}
  
  task_b_labels = {'NGEN':0,'GEN':1}

  return task_a_labels,task_b_labels

### Task A

In [ ]:
train = pd.read_csv('trac2_hin_train.csv')
train.head()

,ID,Text,Sub-task A,Sub-task B
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NAG,NGEN
1,C4.638,Chutiya movie...,NAG,NGEN
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,OAG,NGEN
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,OAG,NGEN
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


time: 53.2 ms (started: 2021-09-03 02:23:24 +00:00)


In [ ]:
train.shape

(3984, 4)

time: 5.45 ms (started: 2021-09-03 02:23:48 +00:00)


In [ ]:
train = train.iloc[:,:-1]

time: 1.71 ms (started: 2021-09-03 02:23:49 +00:00)


In [ ]:
train

,ID,Text,Sub-task A
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NAG
1,C4.638,Chutiya movie...,NAG
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,OAG
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,OAG
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NAG
...,...,...,...
3979,C38.455,Asexual h.. bisexual... homosexual... bhai ase...,NAG
3980,C4.203,Video pura dekne ke pahile hi mai bhai ke vide...,NAG
3981,C45.709,konsa place hai bhai ...nam bolo,NAG
3982,C4.420.1,Kuch zada hi likh diya 🙄,NAG


time: 25 ms (started: 2021-09-03 02:23:50 +00:00)


In [ ]:
for text in train['Text']:
  print(text)

In [ ]:
task_a_labels,_ = labelencoder()
train = train.replace({'Sub-task A':task_a_labels})

time: 6.96 ms (started: 2021-09-03 02:24:02 +00:00)


In [ ]:
train

,ID,Text,Sub-task A
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,0
1,C4.638,Chutiya movie...,0
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,1
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,1
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,0
...,...,...,...
3979,C38.455,Asexual h.. bisexual... homosexual... bhai ase...,0
3980,C4.203,Video pura dekne ke pahile hi mai bhai ke vide...,0
3981,C45.709,konsa place hai bhai ...nam bolo,0
3982,C4.420.1,Kuch zada hi likh diya 🙄,0


time: 27.7 ms (started: 2021-09-03 02:24:04 +00:00)


In [ ]:
labels = train['Sub-task A'].unique()
num_labels = len(labels)

time: 2.91 ms (started: 2021-09-03 02:24:08 +00:00)


In [ ]:
train_texts  = list(train['Text'])
train_labels = list(train['Sub-task A'])

time: 3.29 ms (started: 2021-09-03 02:24:09 +00:00)


In [ ]:
dev = pd.read_csv("trac2_hin_dev.csv")
dev.head()

,ID,Text,Sub-task A,Sub-task B
0,C38.9,bkl interviewers kuch jaada hi open minded bnt...,OAG,NGEN
1,C4.1510,Bhaiya shaadi mein zaroor aana movie ka plot j...,NAG,NGEN
2,C19.95,Section 375 hai kya??? .... Ye to batate kam s...,NAG,NGEN
3,C4.281,कबीर सिंह hit Hui इससे पता चलता है आजकल के लोग...,OAG,NGEN
4,C4.216,Maine itni kam dislike kbhi nhii dekhi,NAG,NGEN


time: 25.7 ms (started: 2021-09-03 02:24:11 +00:00)


In [ ]:
dev.shape

(997, 4)

time: 4.23 ms (started: 2021-09-03 02:24:13 +00:00)


In [ ]:
dev = dev.iloc[:,:-1]

time: 1.61 ms (started: 2021-09-03 02:24:13 +00:00)


In [ ]:
dev = dev.replace({'Sub-task A':task_a_labels})

time: 5.31 ms (started: 2021-09-03 02:24:14 +00:00)


In [ ]:
dev_texts  =  list(dev['Text'])
dev_labels =  list(dev['Sub-task A'])

time: 3.46 ms (started: 2021-09-03 02:24:15 +00:00)


In [ ]:
train_encodings = tokenizer(train_texts,truncation=True)
dev_encodings = tokenizer(dev_texts,truncation=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

time: 387 ms (started: 2021-09-03 02:24:15 +00:00)


In [ ]:
import torch

class TaskDatasetA(torch.utils.data.Dataset):
  
  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels
  
  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item
  
  def __len__(self):
    return len(self.labels)

In [ ]:
train_dataset = TaskDatasetA(train_encodings,train_labels)
dev_dataset = TaskDatasetA(dev_encodings,dev_labels)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments('task-a',evaluation_strategy = 'epoch')

In [ ]:
from transformers import Trainer

In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions = np.argmax(logits,-1)

  acc = accuracy_score(labels,predictions)
  f1  = f1_score(labels,predictions,average='weighted')
  precision = precision_score(labels,predictions,average='weighted')
  recall = precision_score(labels,predictions,average='weighted')

  return {
      'accuracy' : acc,
      'precision' : precision,
      'recall' : recall,
      'f1' : f1
  }

time: 10.1 ms (started: 2021-09-02 17:03:07 +00:00)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    push_to_hub = 'obgv-bert-hi-en'
)

time: 16.8 ms (started: 2021-09-02 17:03:12 +00:00)


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3984
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1494


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.922342,0.682046,0.715986,0.715986,0.692594
2,0.613900,0.779288,0.715145,0.738357,0.738357,0.724133
3,0.498300,1.192768,0.706118,0.706392,0.706392,0.706205


***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to task-a/checkpoint-500
Configuration saved in task-a/checkpoint-500/config.json
Model weights saved in task-a/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to task-a/checkpoint-1000
Configuration saved in task-a/checkpoint-1000/config.json
Model weights saved in task-a/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 997
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1494, training_loss=0.4759446313901439, metrics={'train_runtime': 647.0944, 'train_samples_per_second': 18.47, 'train_steps_per_second': 2.309, 'total_flos': 593986762815264.0, 'train_loss': 0.4759446313901439, 'epoch': 3.0})

time: 10min 47s (started: 2021-09-02 17:03:16 +00:00)


Training time : 10min 47s

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 997
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.7061183550651956,
 'eval_f1': 0.7062054941987269,
 'eval_loss': 1.1927677392959595,
 'eval_precision': 0.7063924274745277,
 'eval_recall': 0.7063924274745277,
 'eval_runtime': 13.0226,
 'eval_samples_per_second': 76.559,
 'eval_steps_per_second': 9.599}

time: 13 s (started: 2021-09-02 17:20:47 +00:00)


Dev set (inference time) :13s

|Logistic Regression|Train | Dev | 
-----------|--------|-----|          
|Accuracy  | 0.67  | 0.69 |
|F1 weighted | 0.66 | 0.67|


|Naive Bayes       |Train | Dev |
-------------------|--------|-----|          
|Accuracy          | 0.70  | 0.69 |
|F1 weighted       | 0.68 | 0.66| 

|rohanrajpal/bert-base-en-hin-cased       |Train | Dev |
-------------------|--------|-----|          
|Accuracy          | 0.71  | 0.70 |
|F1 weighted       | 0.72 | 0.70| 


|Models       |Train time | Dev inference time |
-------------------|--------|-----|          
|LR          | 7s (cv)  | 150ms |
|NB     | 6s (cv) | 200ms| 
|rr/bert-base-en-hin |10.7min |13s |


### Task B

In [ ]:
trainb = pd.read_csv('trac2_hin_train.csv')
trainb.head()

,ID,Text,Sub-task A,Sub-task B
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NAG,NGEN
1,C4.638,Chutiya movie...,NAG,NGEN
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,OAG,NGEN
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,OAG,NGEN
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


In [ ]:
trainb = trainb.iloc[:,[0,1,3]]

In [ ]:
trainb.head()

,ID,Text,Sub-task B
0,C4.131,Bollywood film dekhne ke samay logic ghar mein...,NGEN
1,C4.638,Chutiya movie...,NGEN
2,C38.598,Us jaat bnde ka khene ka matlab tha mar daluga...,NGEN
3,C4.2101.1,@Feminism Is CANCER *un feminist yeh sahi hai ...,NGEN
4,C29.14.2,Amrit Anand अब तो जुड़े ही है उनको बोलो जुड़ने,NGEN


In [ ]:
_,task_b_labels = labelencoder()
trainb = trainb.replace({'Sub-task B':task_b_labels})

In [ ]:
train_texts = list(trainb['Text'])
train_labels = list(trainb['Sub-task B'])
labels = list(trainb['Sub-task B'].unique())
num_labels = len(labels)

In [ ]:
devb = pd.read_csv('trac2_hin_dev.csv')
devb = devb.iloc[:,[0,1,3]]

In [ ]:
devb = devb.replace({'Sub-task B':task_b_labels})

In [ ]:
dev_texts = list(devb['Text'])
dev_labels = list(devb['Sub-task B'])

Test set

In [ ]:
testb = pd.read_csv('trac2_hin_test.csv')
testb_gold = pd.read_csv('trac2_hin_gold_b.csv')
testb.head()

,ID,Text
0,C52.17,ko
1,C52.39,ladkiyon video
2,C52.73,ki video gahrep
3,C60.3,o sadharon video bhai
4,C60.43,ba bhai kyea bola tum moza aaa giea 😌😌😌😂😂😂


In [ ]:
testb_gold = testb_gold.replace({'Sub-task B':task_b_labels})

In [ ]:
test_texts = list(testb['Text'])
test_labels = list(testb_gold['Sub-task B'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

train_encodings = tokenizer(train_texts,truncation=True)
dev_encodings = tokenizer(dev_texts,truncation=True)
test_encodings = tokenizer(test_texts,truncation=True)

In [ ]:
import torch

class TaskDatasetB(torch.utils.data.Dataset):

  def __init__(self,encodings,labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self,idx):
    item = {key: torch.tensor(val[idx]) for key,val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

time: 7.73 ms (started: 2021-09-03 09:10:08 +00:00)


In [ ]:
train_dataset = TaskDatasetB(train_encodings,train_labels)
dev_dataset = TaskDatasetB(dev_encodings,dev_labels)
test_dataset = TaskDatasetB(test_encodings,test_labels)

time: 1.73 ms (started: 2021-09-03 09:10:12 +00:00)


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

time: 1.11 ms (started: 2021-09-03 09:10:12 +00:00)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    'OGBV-gender-bert-hi-en',
    report_to = "wandb",
    run_name = "rr-bert-base-hi-en-3epoch",
    evaluation_strategy = 'epoch',
    save_strategy='epoch',
    num_train_epochs = 3,
    load_best_model_at_end = True,
    push_to_hub='OGBV-gender-bert-hi-en'
)

PyTorch: setting up devices


time: 7.21 ms (started: 2021-09-03 09:11:27 +00:00)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels = num_labels,ignore_mismatched_sizes=True)

loading configuration file https://huggingface.co/rohanrajpal/bert-base-en-hi-codemix-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/268268472edf1b644809420e2ba78728f1365b20cfc54d7ef47a74e5d83362f5.9d230ead58c7011dc1eb755458c02739a07260b282113dd1f586c4dbfb3b7f7d
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_

time: 2.48 s (started: 2021-09-03 09:11:33 +00:00)


In [ ]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score

def compute_metrics(eval_preds):

  logits,labels = eval_preds
  predictions  = np.argmax(logits,axis=-1)

  acc = accuracy_score(labels,predictions)
  precision = precision_score(labels,predictions,average='weighted')
  recall = recall_score(labels,predictions,average='weighted')
  f1 = f1_score(labels,predictions,average='weighted')
  
  return {
     'acc' : acc,
     'precision' : precision,
     'recall' : recall,
     'f1' : f1
  }

time: 5.48 ms (started: 2021-09-03 09:12:25 +00:00)


In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
time: 1.9 s (started: 2021-09-03 09:12:28 +00:00)


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,

    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

Cloning https://huggingface.co/mlkorra/OGBV-gender-bert-hi-en into local empty directory.


time: 5.87 s (started: 2021-09-03 09:12:30 +00:00)


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 3984
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1494
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: mlkorra (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Acc,Precision,Recall,F1
1,No log,0.408167,0.880642,0.866939,0.880642,0.866046
2,0.412000,0.418635,0.881645,0.874280,0.881645,0.877007
3,0.325800,0.438649,0.885657,0.879580,0.885657,0.881931


***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to OGBV-gender-bert-hi-en/checkpoint-498
Configuration saved in OGBV-gender-bert-hi-en/checkpoint-498/config.json
Model weights saved in OGBV-gender-bert-hi-en/checkpoint-498/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to OGBV-gender-bert-hi-en/checkpoint-996
Configuration saved in OGBV-gender-bert-hi-en/checkpoint-996/config.json
Model weights saved in OGBV-gender-bert-hi-en/checkpoint-996/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 997
  Batch size = 8
Saving model checkpoint to OGBV-gender-bert-hi-en/checkpoint-1494
Configuration saved in OGBV-gender-bert-hi-en/checkpoint-1494/config.json
Model weights saved in OGBV-gender-bert-hi-en/checkpoint-1494/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from OGBV-gender-bert-hi-en/checkp

TrainOutput(global_step=1494, training_loss=0.3256958651255412, metrics={'train_runtime': 711.3689, 'train_samples_per_second': 16.801, 'train_steps_per_second': 2.1, 'total_flos': 593981429695680.0, 'train_loss': 0.3256958651255412, 'epoch': 3.0})

time: 11min 51s (started: 2021-09-03 09:12:39 +00:00)


In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 997
  Batch size = 8


{'epoch': 3.0,
 'eval_acc': 0.880641925777332,
 'eval_f1': 0.8660463883623909,
 'eval_loss': 0.4081672728061676,
 'eval_precision': 0.8669389360030555,
 'eval_recall': 0.880641925777332,
 'eval_runtime': 13.9672,
 'eval_samples_per_second': 71.381,
 'eval_steps_per_second': 8.95}

time: 14 s (started: 2021-09-03 09:24:32 +00:00)


In [ ]:
model_path = 'OGBV-gender-bert-hi-en'
#model.save_pretrained(model_path,push_to_hub=True)
#tokenizer.save_pretrained(model_path,push_to_hub=True)

time: 2.69 ms (started: 2021-09-03 09:24:46 +00:00)


In [ ]:
model.push_to_hub('OGBV-gender-bert-hi-en')

Configuration saved in OGBV-gender-bert-hi-en/config.json
Model weights saved in OGBV-gender-bert-hi-en/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.44k/679M [00:00<?, ?B/s]

'https://huggingface.co/mlkorra/OGBV-gender-bert-hi-en/commit/073e6741ece97b29544dd6704d3b6d26f762ef1f'

time: 11min 5s (started: 2021-09-03 09:24:46 +00:00)


In [ ]:
tokenizer.push_to_hub('OGBV-gender-bert-hi-en')

tokenizer config file saved in OGBV-gender-bert-hi-en/tokenizer_config.json
Special tokens file saved in OGBV-gender-bert-hi-en/special_tokens_map.json


'https://huggingface.co/mlkorra/OGBV-gender-bert-hi-en/commit/fa95c084a95ccab42c2bb02a1b43d8b21d1fa59c'

time: 6 s (started: 2021-09-03 09:39:49 +00:00)


In [ ]:
-----------------

In [ ]:
test_preds = trainer.predict(test_dataset=test_dataset)

***** Running Prediction *****
  Num examples = 1200
  Batch size = 8


time: 22.1 s (started: 2021-09-03 09:40:43 +00:00)


In [ ]:
predictions,label_ids,metrics = test_preds

time: 1.79 ms (started: 2021-09-03 09:41:05 +00:00)


In [ ]:
label_ids

array([0, 0, 0, ..., 1, 1, 1])

time: 7.91 ms (started: 2021-09-03 09:51:13 +00:00)


In [ ]:
model.config

BertConfig {
  "_name_or_path": "rohanrajpal/bert-base-en-hi-codemix-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 119547
}

time: 16.3 ms (started: 2021-09-03 09:43:54 +00:00)


In [ ]:
 "id2label": {
    "0": "negative",
    "1": "neutral",
    "2": "positive"
  },

|Logistic Regression|Train | Dev | 
-----------|--------|-----|          
|Accuracy  | 0.88  | 0.60 |
|F1 weighted | 0.86 | 0.46|


|Naive Bayes (with CountVec)     |Train | Dev |
-------------------|--------|-----|          
|Accuracy          | 0.85  | 0.61 |
|F1 weighted       | 0.82 | 0.49| 

|rohanrajpal/bert-base-en-hin-cased       |Train | Dev | Test|
-------------------|--------|-----| ---|         
|Accuracy          | 0.89  | 0.89 | 0.85 |
|F1 weighted       | 0.89 | 0.89| 0.85 |


|Models       |Train time | Dev inference time | Test inference time
-------------------|--------|-----|---|          
|LR          | 2s (cv)  | 200ms | |
|NB     | 1.5s ) | 200ms|  |
|rr/bert-base-en-hin (3) |10.4min |13s |22.5 | 
|rr/bert (5)|18min |13s | |

**dev** inference - 13.5s

**test** inference - 22.5s

NO increase in performance with 5 epochs

### From HF inference api

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("mlkorra/OGBV-gender-bert-hi-en")

model = AutoModelForSequenceClassification.from_pretrained("mlkorra/OGBV-gender-bert-hi-en")

In [ ]:
input_text = test_texts[700]
input_text

'are hat tere bap ke chut tere bhai ka bhosada mc'

In [ ]:
test_tokenized_texts = tokenizer(input_text[:10],truncation=True)

In [ ]:
test_tokenized_texts

{'input_ids': [101, 10301, 11250, 10361, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [ ]:
test_texts[464]

'seriously if somebody accuses me falsely and destroys my life then i also have\nfull right to destroy the life of that bitch..kyunki jhoot rape case hone kai\nbaad bhi in randio ko koi saza nhi milti ..chutia desh aut chutia laws aur\nchutie woh mard jo khade rehte hai khud admio k khillaaf and support karte hai\nsirf ladies ka no matter what...'

In [ ]:
ids = torch.tensor([test_tokenized_texts['input_ids']])

time: 1.55 ms (started: 2021-09-03 10:04:12 +00:00)


In [ ]:
ids.shape

torch.Size([1, 21])

time: 6.6 ms (started: 2021-09-03 10:04:50 +00:00)


In [ ]:
output = model(ids)
output

SequenceClassifierOutput([('logits',
                           tensor([[-1.0878,  1.9721]], grad_fn=<AddmmBackward>))])

time: 119 ms (started: 2021-09-03 10:05:02 +00:00)


In [ ]:
res = torch.nn.functional.softmax(output.logits,-1)
res

tensor([[0.0448, 0.9552]], grad_fn=<SoftmaxBackward>)

time: 8 ms (started: 2021-09-03 09:53:13 +00:00)


In [ ]:
predictions = np.argmax(output.logits.detach().numpy(),-1)

time: 12.8 ms (started: 2021-09-03 09:55:21 +00:00)


In [ ]:
predictions[0]

1

time: 5.78 ms (started: 2021-09-03 09:55:34 +00:00)


test evalution on a pair of sentences

In [ ]:
input_texts = test_texts[:2]
input_texts

['ko', 'ladkiyon video']

time: 5.38 ms (started: 2021-09-03 10:05:10 +00:00)


In [ ]:
test_tokenized_texts = tokenizer(input_texts,truncation=True,padding=True)

time: 3.24 ms (started: 2021-09-03 10:05:10 +00:00)


In [ ]:
ids_1 = torch.tensor(test_tokenized_texts['input_ids'])

time: 1.79 ms (started: 2021-09-03 10:05:55 +00:00)


In [ ]:
ids_1

tensor([[  101, 11252,   102,     0,     0,     0,     0],
        [  101, 10109, 10162, 10506, 15820, 12078,   102]])

time: 5.26 ms (started: 2021-09-03 10:05:57 +00:00)


In [ ]:
ids_1.shape

torch.Size([2, 7])

time: 9.12 ms (started: 2021-09-03 10:06:00 +00:00)


In [ ]:
output = model(ids_1)

time: 107 ms (started: 2021-09-03 10:06:07 +00:00)


In [ ]:
output

SequenceClassifierOutput([('logits', tensor([[ 2.2235, -1.4354],
                                   [ 1.9103, -1.0534]], grad_fn=<AddmmBackward>))])

time: 9.38 ms (started: 2021-09-03 10:07:47 +00:00)


In [ ]:
res = torch.nn.functional.softmax(output.logits,-1)
res

tensor([[0.9749, 0.0251],
        [0.9509, 0.0491]], grad_fn=<SoftmaxBackward>)

time: 12.5 ms (started: 2021-09-03 10:08:19 +00:00)


In [ ]:
preds = np.argmax(output.logits.detach().numpy(),-1)
preds

array([0, 0])

time: 6.39 ms (started: 2021-09-03 10:09:18 +00:00)


test evaluation on all sentences

In [ ]:
test_texts 

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
predictions = []
for input_text in tqdm(test_texts):
  tokenized_text = tokenizer(input_text,truncation=True)
  input_ids = torch.tensor(tokenized_texts['input_ids'])
  output = model(input_ids)
  preds = np.argmax(output.logits.detach().numpy(),-1)
  predictions.append(preds[0])

100%|██████████| 1200/1200 [25:50<00:00,  1.29s/it]


In [ ]:
task_b_preds = pd.DataFrame(predictions,columns=['Sub-Task B predictions'])
task_b_preds.to_csv("TaskBpredictions.csv")

In [ ]:
!ls

obgv-gender-bert-hi-en	TaskBpredictions.csv  trac2_hin_test.csv
OGBV-gender-bert-hi-en	trac2_hin_dev.csv     trac2_hin_train.csv
sample_data		trac2_hin_gold_b.csv  wandb


In [ ]:
from google.colab import files

In [ ]:
files.download('TaskBpredictions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>